In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle

import warnings
warnings.filterwarnings("ignore")
SIZE = 512

In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

# POS_WEIGHT = 1.0  # multiplier for positive targets, needs to be tuned

# def weighted_binary_crossentropy(target, output):
#     """
#     Weighted binary crossentropy between an output tensor 
#     and a target tensor. POS_WEIGHT is used as a multiplier 
#     for the positive targets.

#     Combination of the following functions:
#     * keras.losses.binary_crossentropy
#     * keras.backend.tensorflow_backend.binary_crossentropy
#     * tf.nn.weighted_cross_entropy_with_logits
#     """
#     # transform back to logits
# #     _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
#     _epsilon = K.epsilon()
#     output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
#     output = tf.log(output / (1 - output))
#     # compute weighted loss
#     loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
#                                                     logits=output,
#                                                     pos_weight=POS_WEIGHT)
#     return tf.reduce_mean(loss, axis=-1)

# import tensorflow as tf
# from tensorflow.python.framework import ops
# from functools import reduce

# def binaryRound(x):
#     """
#     Rounds a tensor whose values are in [0,1] to a tensor with values in {0, 1},
#     using the straight through estimator for the gradient.
#     """
#     g = tf.get_default_graph()

#     with ops.name_scope("BinaryRound") as name:
#         with g.gradient_override_map({"Round": "Identity"}):
#             return tf.round(x, name=name)

#         # For Tensorflow v0.11 and below use:
#         #with g.gradient_override_map({"Floor": "Identity"}):
#         #    return tf.round(x, name=name)
        
# def f1_loss2(y_true, y_pred):
#     y_pred = binaryRound(y_pred)
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1-K.mean(f1))

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [7]:
train_dataset_info

array([ {'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}], dtype=object)

In [8]:
submit = pd.read_csv('../data/sample_submission.csv')
path_to_test = '../data/test/'
test_dataset_info = []
for name in tqdm(submit['Id']):
    test_dataset_info.append({
        'path':os.path.join(path_to_test, name)})
test_dataset_info = np.array(test_dataset_info)

100%|██████████| 11702/11702 [00:00<00:00, 198987.85it/s]


In [9]:
# rgb_arr = np.memmap('../cache/tmp_rgb_arr', dtype='uint8', mode='r+', 
#                    shape=(len(train_dataset_info),299,299,3))

In [10]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels
    def create_train2(dataset_info, batch_size, shape, augument=False):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    
                    batch_images.append(image/255.)
                    
                yield np.array(batch_images, np.float32), None
    def create_test(dataset_info, batch_size, shape, augument=False):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_test_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_test_batch), 28))
                for i in range(len(X_test_batch)):
                    image = data_generator.load_image(
                        X_test_batch[i]['path'], shape) 
#                     image = data_generator.load_image(
#                         i, shape) 
                    
                    batch_images.append(image/255.)
                    
                yield np.array(batch_images, np.float32), None
    def load_image(path, shape):
        img1 = cv2.imread(path+'_red.png', cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(path+'_green.png', cv2.IMREAD_GRAYSCALE)
        img3 = cv2.imread(path+'_blue.png', cv2.IMREAD_GRAYSCALE)
        image = np.stack((img1,img2,img3), -1)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    def load_image3(idx, shape):
#         print(idx)
        name = '../cache/RGB/img-{}.png'.format(idx)
        image = cv2.imread(name)
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    def load_image2(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
#         zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_yellow_ch),
#             zero_data, zero_data), -1)
#         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2))
        image = cv2.resize(image, (shape[0], shape[1]))
        return image
    
    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug


In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.layers import UpSampling2D, Lambda, Reshape, Conv2DTranspose
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

Using TensorFlow backend.


In [12]:
def decoder_block(x,blocks=6,start_filters=512):
    for i in range(blocks):
        x = Conv2D(start_filters//(2**i),(3,3), activation='relu', padding='same')(x)
        x = Conv2D(start_filters//(2**i),(3,3), activation='relu', padding='same')(x)
        x = UpSampling2D()(x)
    return x

In [13]:
model = decoder_block(x = Input(shape=(512,512,3)))
model.shape

TensorShape([Dimension(None), Dimension(32768), Dimension(32768), Dimension(16)])

In [14]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon


In [70]:
img_shape = (512, 512, 3)
batch_size = 16
latent_dim = 2


input_img = Input(shape=img_shape)

x = Conv2D(32, 3,padding='same', activation='relu')(input_img)
x = Conv2D(64, 3,padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2D(64, 3,padding='same', activation='relu')(x)
x = Conv2D(64, 3,padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)
z = Lambda(sampling)([z_mean, z_log_var])

print(z.shape)
# This is the input where we will feed `z`.
decoder_input = Input(K.int_shape(z)[1:])

print(decoder_input.shape)
# Upsample to the correct number of units
x = Dense(np.prod(shape_before_flattening[1:]),
             activation='relu')(decoder_input)

print(x.shape)
# Reshape into an image of the same shape as before our last `Flatten` layer
x = Reshape(shape_before_flattening[1:])(x)

print(shape_before_flattening)
print(x.shape)
# We then apply then reverse operation to the initial
# stack of convolution layers: a `Conv2DTranspose` layers
# with corresponding parameters.
x = Conv2DTranspose(32, 3,
                           padding='same', activation='relu',
                           strides=(2, 2))(x)

print(x.shape)
x = Conv2D(3, 3,
                  padding='same', activation='sigmoid')(x)
# We end up with a feature map of the same size as the original input.
print(x.shape)
# This is our decoder model.
decoder = Model(decoder_input, x)

# We then apply it to `z` to recover the decoded `z`.
z_decoded = decoder(z)
# return z_decoded



(?, 2)
(?, 2)
(?, 4194304)
(None, 256, 256, 64)
(?, 256, 256, 64)
(?, ?, ?, 32)
(?, ?, ?, 3)


In [72]:
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        # We don't use this output.
        return x

# We call our custom layer on the input and the decoded output,
# to obtain the final model output.
y = CustomVariationalLayer()([input_img, z_decoded])

In [17]:
vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 512, 512, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 256, 256, 64) 18496       conv2d_13[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_16 

In [18]:
# batch_size = 16
# indexes = np.arange(train_dataset_info.shape[0])
# train_generator = data_generator.create_train2(
#     train_dataset_info[indexes], batch_size, (SIZE,SIZE,3), augument=True)

In [19]:
l_train = len(train_dataset_info) 
l_test = 11702

In [20]:
len_all = l_train + l_test
len_all

42774

In [21]:
# X_rgb = np.memmap('../cache/rgb_arr_512_512', dtype=np.float32, mode='w+', shape=(len_all, 512,512,3))

In [22]:
# for ii in tqdm(range(l_train)):
#     path = train_dataset_info[ii]['path']
#     image = data_generator.load_image(path, (SIZE,SIZE,3))
#     X_rgb[ii] = image


In [23]:
# submit = pd.read_csv('../data/sample_submission.csv')
# for idx, name in tqdm(enumerate(submit['Id'])):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))
#     X_rgb[idx+l_train] = image

In [24]:
# X_rgb = np.memmap('../cache/rgb_arr_512_512', dtype=np.float32, mode='r+', shape=(len_all, 512,512,3))

In [25]:
# from keras.datasets import mnist
# (x_train, _), (x_test, y_test) = mnist.load_data()


In [26]:
# x_train.shape, x_test.shape, y.shape

In [27]:
# x_train = X_rgb[:l_train].copy()
# x_test = X_rgb[l_train:].copy()
# x_train.shape, x_test.shape

In [28]:
# batch_size = 16

# x_train = x_train.astype('float32') / 255.
# # x_train = x_train.reshape(x_train.shape + (1,))
# x_test = x_test.astype('float32') / 255.
# # x_test = x_test.reshape(x_test.shape + (1,))



In [29]:
# vae.fit(x=x_train, y=None,
#         shuffle=True,
#         epochs=25,
#         batch_size=batch_size,
#         validation_data=(x_test, None))

In [30]:
# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
# train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)
train_indexes = indexes
valid_indexes = np.arange(l_test)
# create train and valid datagens
train_generator = data_generator.create_train2(
    train_dataset_info[train_indexes], 32, (SIZE,SIZE,3), augument=True)
validation_generator = data_generator.create_test(
    test_dataset_info[valid_indexes], 16, (SIZE,SIZE,3), augument=False)

In [31]:
batch_size=32
vae.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / 16.),
    epochs=8, 
    verbose=1)

Epoch 1/8
971/971 [==============================] - 1108s 1s/step - loss: 2048073432.9487 - val_loss: 0.1828
Epoch 2/8
971/971 [==============================] - 806s 830ms/step - loss: 0.2228 - val_loss: 0.1820
Epoch 3/8
971/971 [==============================] - 798s 822ms/step - loss: 0.2220 - val_loss: 0.1818
Epoch 4/8
971/971 [==============================] - 795s 819ms/step - loss: 0.2217 - val_loss: 0.1821
Epoch 5/8
971/971 [==============================] - 803s 827ms/step - loss: 0.2215 - val_loss: 0.1810
Epoch 6/8
971/971 [==============================] - 796s 820ms/step - loss: 0.2214 - val_loss: 0.1822
Epoch 7/8
971/971 [==============================] - 790s 814ms/step - loss: 0.2213 - val_loss: 0.1813
Epoch 8/8
971/971 [==============================] - 831s 856ms/step - loss: 0.2212 - val_loss: 0.1810


In [32]:
# vae.save_weights('../cache/vae-33-1.h5')

In [1]:
# import matplotlib.pyplot as plt
# from scipy.stats import norm

# # Display a 2D manifold of the digits
# n = 15  # figure with 15x15 digits
# digit_size = 512
# figure = np.zeros((digit_size * n, digit_size * n, 3))
# # Linearly spaced coordinates on the unit square were transformed
# # through the inverse CDF (ppf) of the Gaussian
# # to produce values of the latent variables z,
# # since the prior of the latent space is Gaussian
# grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
# grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

# for i, yi in enumerate(grid_x):
#     for j, xi in enumerate(grid_y):
#         z_sample = np.array([[xi, yi]])
#         z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
#         x_decoded = decoder.predict(z_sample, batch_size=batch_size)
#         digit = x_decoded[0].reshape(digit_size, digit_size, 3)
#         figure[i * digit_size: (i + 1) * digit_size,
#                j * digit_size: (j + 1) * digit_size] = digit

# plt.figure(figsize=(10, 10))
# plt.imshow(figure, cmap='Greys_r')
# plt.show()

In [ ]:
def global_average_pooling(x):
    return K.mean(x, axis = (2, 3))
    
def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

In [ ]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(256, kernel_size=(1,1), activation='relu', name='conv2')(x)
    
#     x = Conv2D(32, kernel_size=(1,1), activation='relu', name='conv1')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [15]:
# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#         pt_1 = K.clip(pt_1, 1e-3, .999)
#         pt_0 = K.clip(pt_0, 1e-3, .999)

#         return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed

In [16]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../cache/InceptionV3-34.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [17]:

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)

# create train and valid datagens
train_generator = data_generator.create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = data_generator.create_train(
    train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)


In [18]:
# warm up model
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=28)
model.summary()

for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
batch_normalization_95 (Batc (None, 512, 512, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 14, 14, 2048)      21802784  
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 14, 256)       524544    
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              51381248  
__________

In [19]:
model.compile(
    loss=f1_loss, 
    optimizer=Adam(1e-03),
    metrics=[f1])


In [20]:
batch_size=8
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

Epoch 1/2
3302/3302 [==============================] - 1135s 344ms/step - loss: 1.1137 - f1: 0.0464 - val_loss: 1.5707 - val_f1: 0.0334
Epoch 2/2
3302/3302 [==============================] - 1029s 312ms/step - loss: 1.0977 - f1: 0.0620 - val_loss: 1.4721 - val_f1: 0.0346


In [21]:
epochs=120

for layer in model.layers:
    layer.trainable = True
model.compile(loss=f1_loss,
            optimizer=Adam(lr=1e-4),
            metrics=[f1])

In [22]:
# train all layers
batch_size=4
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=epochs, 
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/120
6603/6603 [==============================] - 3791s 574ms/step - loss: 0.9487 - f1: 0.1932 - val_loss: 0.8098 - val_f1: 0.3203

Epoch 00001: val_loss improved from inf to 0.80976, saving model to ../cache/InceptionV3-8i1.h5
Epoch 2/120
6603/6603 [==============================] - 3784s 573ms/step - loss: 0.8446 - f1: 0.2700 - val_loss: 0.7620 - val_f1: 0.3501

Epoch 00002: val_loss improved from 0.80976 to 0.76196, saving model to ../cache/InceptionV3-8i1.h5
Epoch 3/120
6603/6603 [==============================] - 3752s 568ms/step - loss: 0.8079 - f1: 0.2944 - val_loss: 0.7483 - val_f1: 0.3740

Epoch 00003: val_loss improved from 0.76196 to 0.74834, saving model to ../cache/InceptionV3-8i1.h5
Epoch 4/120
6603/6603 [==============================] - 3773s 571ms/step - loss: 0.7817 - f1: 0.3105 - val_loss: 0.7292 - val_f1: 0.3805

Epoch 00004: val_loss improved from 0.74834 to 0.72923, saving model to ../cache/InceptionV3-8i1.h5
Epoch 5/120
6603/6603 [=========================

In [23]:
# for ii in tqdm(np.arange(len(train_dataset_info))):
#     img1 = cv2.imread(train_dataset_info[ii]['path']+'_red.png', cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(train_dataset_info[ii]['path']+'_green.png', cv2.IMREAD_GRAYSCALE)
#     img3 = cv2.imread(train_dataset_info[ii]['path']+'_blue.png', cv2.IMREAD_GRAYSCALE)
#     img1 = np.stack((img1, img2, img3), -1)
#     name = '../cache/RGB/img-{}.png'.format(ii)
#     cv2.imwrite(name,img1)

In [24]:
# submit = pd.read_csv('../data/sample_submission.csv')
# for name in tqdm(submit['Id']):
#     path = os.path.join('../data/test/', name)
#     img1 = cv2.imread(path+'_red.png', cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(path+'_green.png', cv2.IMREAD_GRAYSCALE)
#     img3 = cv2.imread(path+'_blue.png', cv2.IMREAD_GRAYSCALE)
#     img1 = np.stack((img1, img2, img3), -1)
#     name1 = '../cache/RGB/test/img-{}.png'.format(name)
#     cv2.imwrite(name1,img1)

In [25]:
# Create submit
submit = pd.read_csv('../data/sample_submission.csv')
predicted = []
draw_predict = []
# model = create_model(
#     input_shape=(SIZE,SIZE,3), 
#     n_out=28)
# for layer in model.layers:
#     layer.trainable = True
# model.compile(loss=f1_loss,
#             optimizer=Adam(lr=1e-4),
#             metrics=[f1])
model.load_weights('../cache/InceptionV3-8i1.h5')
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-8.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

100%|██████████| 11702/11702 [08:48<00:00, 22.16it/s]


In [26]:
submit.to_csv('../submissions/sub8i1-a.csv', index=False)

In [28]:
predicted = []
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.25]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub8i1-b.csv', index=False)

100%|██████████| 11702/11702 [08:59<00:00, 21.70it/s]


In [29]:
predicted = []
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.3]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit.to_csv('../submissions/sub8i1-c.csv', index=False)

100%|██████████| 11702/11702 [09:18<00:00, 20.97it/s]


In [30]:
predicted = []
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.35]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub8i1-d.csv', index=False)

100%|██████████| 11702/11702 [09:26<00:00, 20.66it/s]


In [31]:
predicted = []
for name in tqdm(submit['Id']):
    path = os.path.join('../data/test/', name)
    image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
    score_predict = model.predict(image[np.newaxis])[0]
    draw_predict.append(score_predict)
    label_predict = np.arange(28)[score_predict>=0.4]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub8i1-e.csv', index=False)

100%|██████████| 11702/11702 [09:29<00:00, 20.54it/s]


In [32]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [33]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-11-19 12:33:39.384598


In [28]:
# %%time
# !kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub8i1.csv -m ""

100%|████████████████████████████████████████| 480k/480k [00:08<00:00, 51.3kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 332 ms, sys: 134 ms, total: 467 ms
Wall time: 15.6 s


In [29]:
# from time import sleep
# sleep(60)
# !kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub8i.csv     2018-11-14 04:28:35               complete  0.443        None          
sub8h.csv     2018-11-13 23:48:00               complete  0.434        None          
sub8g.csv     2018-11-13 07:17:59               complete  0.389        None          
sub8c.csv     2018-11-11 14:31:02               complete  0.429        None          
sub30.csv     2018-11-09 07:02:56               complete  0.033        None          
sub29.csv     2018-11-08 22:07:11               complete  0.389        None          
sub28-c.csv   2018-11-08 15:47:08               complete  0.457        None          
sub28-bb.csv  2018-11-08 15:46:13               complete  0.458        None          
sub28-b.csv   2018-11-08 15:45:28               complete  0.454        None          
sub28-a.csv   2018-11-08 15:44:27               comple

In [26]:
from time import sleep
sleep(60)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName  date                 description  status    publicScore  privateScore  
--------  -------------------  -----------  --------  -----------  ------------  
sub8.csv  2018-10-20 20:08:45               complete  0.422        None          
sub7.csv  2018-10-20 17:06:09               complete  0.389        None          
sub5.csv  2018-10-19 18:27:33               complete  0.387        None          
sub4.csv  2018-10-19 14:45:15               complete  0.411        None          
sub3.csv  2018-10-19 10:19:26               complete  0.377        None          
sub2.csv  2018-10-19 08:07:30               complete  0.135        None          
sub1.csv  2018-10-19 06:28:57               complete  0.374        None          
